## Создание датафрейма

In [65]:
import json
import re

import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from io import BytesIO

auth.authenticate_user()
drive_service = build('drive', 'v3')


In [66]:
json_list = [
    '1nLCqQrgkMWX4l3aVxBhZrOopdXDho1vz',
    '1VAmuBxlFB2PJcoLrPI8KHgXLgskhnhSh',
    '1FCPe9XIYsp0G_paTiXuzWz37Bq3vj_bW',
    '1il2rUMH69jp0KopUL16k--RAyzvxELf4',
    '1frN4UgyVsJGKhX0itEZeYTueffVRVuY5'
    ]

In [67]:
def text_to_str(cell):
  txt = cell
  textstr = ''
  if type(txt) == list:
    for e in txt:
      if type(e) == str:
        textstr += e
      elif type(e) == dict:
        textstr += e['text']
    return re.findall(r"[а-яА-ЯёЁa-zA-Z']+", textstr.lower())
  elif type(txt) == str:
    return [re.findall(r"[а-яА-ЯёЁa-zA-Z']+", txt.lower())]
  else:
    return ['not found']

In [68]:
mediadf = pd.DataFrame(columns=['date_unixtime', 'from', 'from_id', 'text', 'text_list'])

for media in json_list:
  file_id = media
  request = drive_service.files().get_media(fileId=file_id)
  content = request.execute()

  data = json.loads(content.decode('utf-8'))
  df = pd.DataFrame(data['messages'])[['date_unixtime', 'from', 'from_id', 'text']]
  df['text_list'] = df['text'].apply(text_to_str)
  mediadf = pd.concat([mediadf, df])

In [26]:
synonym_dict = {
    # Действующие власти и системные политики
    'Путин': 'путин',
    'Медведев': 'медведев',
    'Мишустин': 'мишустин',
    'Шойгу': 'шойгу',
    'Лавров': 'лавров',
    'Патрушев': 'патрушев',
    'Володин': 'володин',
    'Матвиенко': 'матвиенко',
    'Зюганов': 'зюганов',
    'Жириновский': 'жириновский',
    'Миронов': 'миронов',
    'Собянин': 'собянин',
    'Кадыров': 'кадыров',
    'Колокольцев': 'колокольцев',
    'Кириенко': 'кириенко',
    'Песков': 'песков',
    'Бастрыкин': 'бастрыкин',
    'Герасимов': 'герасимов',
    'Бортников': 'бортников',
    'Мантуров': 'мантуров',
    'Скворцова': 'скворцова',
    'Кобылкин': 'кобылкин',
    'Клишас': 'клишас',
    'Яровая': 'яровая',
    'Беглов': 'беглов',
    'Куйвашев': 'куйвашев',
    'Текслер': 'текслер',
    'Артамонов': 'артамонов',
    'Радаев': 'радаев',
    'Азаров': 'азаров',

    # Оппозиционные и несистемные деятели
    'Навальный': 'навальный',
    'Яшин': 'яшин',
    'Соболь': 'соболь',
    'Волков': 'волков',
    'Жданов': 'жданов',
    'Кац': 'кац',
    'Гудков': 'гудков',
    'Гудкова': 'гудкова',
    'Галямина': 'галямина',
    'Ройзман': 'ройзман',
    'Шендерович': 'шендерович',
    'Ходорковский': 'ходорковский',
    'Каспаров': 'каспаров',
    'Пономарев': 'пономарев',
    'Удальцов': 'удальцов',
    'Фургал': 'фургал',
    'Кара-Мурза': 'кара-мурза',
    'Алексеева': 'алексеева',
    'Чирикова': 'чирикова',
    'Бондарь': 'бонарь',
    'Албуров': 'албуров',
    'Сторчак': 'сторчак',
    'Долматов': 'долматов',
    'Мальцев': 'мальцев',
    'Собчак': 'собчак',
    'Габрелянов': 'габрелянов',

    # Региональные и другие известные фигуры
    'Кожин': 'кожин',
    'Ткачев': 'ткачев',
    'Полтавченко': 'полтавченко',
    'Дюмин': 'дюмин',
    'Воробьев': 'воробьев',
    'Козак': 'козак',
    'Шахрай': 'шахрай',
    'Голодец': 'голодец',
    'Решетников': 'решетников',
    'Орешкин': 'орешкин',
    'Силуанов': 'силуанов',
    'Новак': 'новак',
    'Карасин': 'карасин',
    'Небензя': 'небензя',
    'Зорькин': 'зорькин',
    'Чайка': 'чайка',
    'Мизулина': 'мизулина',
    'Пушков': 'пушков',
    'Слуцкий': 'слуцкий',
    'Журова': 'журова',
    'Торлопов': 'торлопов',
    'Морозов': 'морозов',
    'Климов': 'климов',
    'Круглов': 'круглов',
    'Громов': 'громов',
    'Шевченко': 'шевченко',
    'Калашников': 'калашников',
    'Маргелов': 'маргелов',
    'Фалин': 'фалин',
    'Скабеева': 'скабеева',
    'Платошкин': 'платошкин',
    'Кургинян': 'кургинян',
    'Проханов': 'проханов',
    'Леонтьев': 'леонтьев',

    # Украинские политики (после 2014 г.)
    'Зеленский': 'зеленский',
    'Шмыгаль': 'шмыгаль',
    'Ермак': 'ермак',
    'Арестович': 'арестович',
    'Порошенко': 'порошенко',
    'Тимошенко': 'тимошенко',
    'Кличко': 'кличко',
    'Залужный': 'залужный',
    'Резников': 'резников',
    'Данилов': 'данилов',
    'Медведчук': 'медведчук',
    'Бужанский': 'бужанский',
    'Мураев': 'мураев',
    'Янукович': 'янукович',
    'Аваков': 'аваков',
    'Геращенко': 'геращенко',
    'Киселевский': 'киселевский',  # Министр инфраструктуры (2023)
    'Стефанчук': 'стефанчук',  # Глава Верховной Рады
    'Маркарова': 'маркарова',  # Экс-посол Украины в США
    'Дубинский': 'дубинский',  # Нардеп, фигурант скандалов
    'Труханов': 'труханов',  # Мэр Одессы
    'Садовый': 'садовый'  # Мэр Львова
}

In [28]:
def find_matching_keys(cell):
    """
    Принимает список слов и возвращает набор ключей из словаря synonym_dict,
    чьи синонимы являются началом хотя бы одного слова в списке.
    """
    matched_keys = set()
    for word in cell:
        if type(word) == str:
            lower_word = word.lower()
            for key, synonym in synonym_dict.items():
                if lower_word.startswith(synonym.lower()):
                    matched_keys.add(key)
        elif type(word) == list:
          for wordd in word:
            if type(wordd) == str:
                lower_word = wordd.lower()
                for key, synonym in synonym_dict.items():
                    if lower_word.startswith(synonym.lower()):
                        matched_keys.add(key)
    return matched_keys

mediadf['matched_keys'] = mediadf['text_list'].apply(find_matching_keys)
mediadf = mediadf.sort_values(by='date_unixtime')

In [30]:
mediadf

,date_unixtime,from,from_id,text,text_list,matched_keys
0,1704056426,Readovka,channel1260622817,"[{'type': 'bold', 'text': '«Мы едины в труде и...","[мы, едины, в, труде, и, в, бою, владимир, пут...",{Путин}
0,1704056463,РИА Новости,channel1101170442,Новогоднее обращение Владимира Путина,"[[новогоднее, обращение, владимира, путина]]",{Путин}
0,1704056554,РБК. Новости. Главное,channel1099860397,"[Президент Владимир Путин , {'type': 'text_lin...","[президент, владимир, путин, выступил, с, ново...",{Путин}
1,1704056721,Readovka,channel1260622817,"[{'type': 'bold', 'text': 'Карта военных дейст...","[карта, военных, действий, и, ситуация, на, фр...",{}
1,1704056866,РБК. Новости. Главное,channel1099860397,"[🎄 Дорогие читатели, поздравляем вас с Новым г...","[дорогие, читатели, поздравляем, вас, с, новым...",{}
...,...,...,...,...,...,...
68988,1735588946,ТАСС,channel1050820672,Из-за ракетного обстрела Льгова повреждения по...,"[[из, за, ракетного, обстрела, льгова, поврежд...",{}
48609,1735589024,РИА Новости,channel1101170442,"Национальная сирийская конференция, где будет ...","[[национальная, сирийская, конференция, где, б...",{}
68989,1735589130,ТАСС,channel1050820672,Газоснабжение временно отключено в поврежденно...,"[[газоснабжение, временно, отключено, в, повре...",{}
48610,1735589224,РИА Новости,channel1101170442,[Евросоюз готов к остановке транзита российско...,"[евросоюз, готов, к, остановке, транзита, росс...",{}


In [29]:
mediadf.to_csv('total_media.csv')

### Описание получившегося датафрейма

*   date_unixtime	- Дата сообщения в unix формате
*   from - Название канала
*   from_id	- ID канала
*   text - текст сообщения (как в json)
*   text_list	- Список слов из сообщения в нижнем регистре
*   matched_keys - Набор деятелей, упоминаемых в сообщении

## Исследование данных

In [57]:
df = mediadf

### Частотная таблица
Основная цель - выявить самый упоминаемых деятелей

In [58]:
# Предположим, df — ваш исходный датафрейм
# Преобразуем множества в списки (если необходимо)
df['matched_keys'] = df['matched_keys'].apply(lambda x: list(x))

# Разворачиваем колонку matched_keys
df_exploded = df.explode('matched_keys')

# Считаем частоту упоминаний
frequency_table = df_exploded['matched_keys'].value_counts().reset_index()
frequency_table.columns = ['Деятель', 'Кол-во упоминаний']

frequency_table

,Деятель,Кол-во упоминаний
0,Путин,15245
1,Песков,2974
2,Лавров,1817
3,Зеленский,1365
4,Медведев,1009
...,...,...
95,Проханов,3
96,Ройзман,2
97,Галямина,1
98,Полтавченко,1


In [59]:
# Сортируем и берем топ-10
top_10 = frequency_table.head(10).sort_values('Кол-во упоминаний', ascending=False)

# Создаем бар-чарт
fig = px.bar(
    top_10,
    x='Деятель',
    y='Кол-во упоминаний',
    title='Топ-10 самых упоминаемых деятелей',
    color='Кол-во упоминаний',  # Добавим цветовую шкалу
    text='Кол-во упоминаний',   # Покажем значения на столбцах
    color_continuous_scale='Bluered'  # Выбор цветовой гаммы
)

# Настраиваем отображение
fig.update_layout(
    xaxis_title='Деятель',
    yaxis_title='Количество упоминаний',
    hovermode='x unified',      # Показывать данные при наведении
    showlegend=False            # Скрываем легенду
)

# Настраиваем текст
fig.update_traces(
    texttemplate='%{text}',
    textposition='outside'
)

# Показать график
fig.show()


In [62]:
# Преобразование UNIX времени с обработкой типа данных
df['date_unixtime'] = pd.to_numeric(df['date_unixtime'])
df['date'] = pd.to_datetime(df['date_unixtime'], unit='s')

# Разворачиваем множества ключей
exploded_df = df.explode('matched_keys')

# Создаем недельные метки
exploded_df['week_start'] = exploded_df['date'].dt.to_period('W').dt.start_time

# Создаем полный диапазон недель
all_weeks = pd.date_range(
    start=exploded_df['week_start'].min(),
    end=exploded_df['week_start'].max(),
    freq='W-MON'
)

# Группируем и заполняем пропуски
time_series = (
    exploded_df.groupby(['week_start', 'matched_keys'])
    .size()
    .unstack(fill_value=0)
    .reindex(all_weeks, fill_value=0)
    .stack()
    .reset_index(name='count')
)

# Переименовываем колонку для совместимости
time_series.rename(columns={'level_0': 'week_start'}, inplace=True)

# Выбираем топ-10 ключевых слов
top_10_words = (
    time_series
    .groupby('matched_keys')['count']
    .sum()
    .nlargest(10)
    .index
)

# Фильтруем данные только для топ-10
time_series_top10 = time_series[time_series['matched_keys'].isin(top_10_words)]

# Обновляем категории для правильной сортировки
time_series_top10['matched_keys'] = pd.Categorical(
    time_series_top10['matched_keys'],
    categories=top_10_words,
    ordered=True
)

# Сортировка данных
time_series_top10 = time_series_top10.sort_values(['week_start', 'matched_keys'])

# Построение графика
fig = px.line(
    time_series_top10,
    x='week_start',
    y='count',
    color='matched_keys',
    title='Интерактивная динамика упоминаемости деятелей',
    labels={'count': 'Количество упоминаний', 'week_start': 'Неделя'},
    markers=True
)

# Настройка отображения
fig.update_layout(
    hovermode='x unified',
    xaxis_title='Неделя',
    yaxis_title='Количество упоминаний',
    legend_title='Ключевые слова',
    template='plotly_white'
)

# Корректные элементы управления
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(step="all")
        ])
    ),
    tickformat="%d %b %Y"
)

fig.show()

<ipython-input-62-cb2aeec95e57>:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
frequency_table.to_csv('top100.csv', index=False)